# Covid19 Data Processing
1. Data source is [John Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)
1. Convert wide format to long format and sanitize the data
1. Inputs are three files: `confimred`, `deaths`, and `recovered`
1. Ouput four files:
    1. `covid19-confirmed-<date>.csv`
    1. `covid19-deaths-<date>.csv`
    1. `covid19-recovered-<date>.csv`
    1. `covid19-<date>.csv` - contains confirmed, deaths and recovered
    1. `covid19-<date>.csv` - contains confirmed, deaths and recovered in the long format i.e. (type, count)
    1. `covid19.c6p` - geo-coordinates, i.e State to latlong

In [132]:
import pandas as pd

In [169]:
confirmed = pd.read_csv('../data-jhu/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
deaths = pd.read_csv('../data-jhu/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
recovered = pd.read_csv('../data-jhu/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [170]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,70,75,82,114,147,177,212,272,322,411
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,639,701,773,839,825,878,889,924,963,1007
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,178,200,212,226,243,266,313,345,385,432
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,149,197,238,428,566,673,790,900,1030,1183


# Convert wide format to long format

In [177]:
def wide_to_long(df, col, pos=4, var_name='Date'):
    """
    pos: 0-based, start index of the fields that need to be converted to long format
    """
    return pd.melt(df, id_vars=df.columns[:pos], value_vars=df.columns[pos:], var_name=var_name, value_name=col)

# Sanitize data

In [178]:
def sanitize(df, col):
    df = wide_to_long(df, col=col)
    
    # rename
    df.rename(columns={'Province/State': 'Province_State', 'Country/Region':'Country_Region'}, inplace=True)
    
    # remove comma
    df['Country_Region']=df['Country_Region'].str.replace(',', '')
    df['Province_State']=df['Province_State'].str.replace(',', '')
    
    # change date format
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')
    
    return df;

In [179]:
confirmed = sanitize(confirmed, 'ConfirmedCases')
deaths = sanitize(deaths, 'Fatalities')
recovered = sanitize(recovered, 'RecoveredCases')

In [180]:
def fill_state(df, src='Country_Region', tgt='Province_State'):
    """
    Use the value in `src` column to fill the `tgt` column if `tgt` is NaN
    """
    idx = df[tgt].isnull()
    df.loc[idx, tgt] = df[src].loc[idx]

In [181]:
fill_state(confirmed)
fill_state(deaths)
fill_state(recovered)

In [182]:
confirmed.shape

(28920, 6)

In [183]:
confirmed.head()

,Province_State,Country_Region,Lat,Long,Date,ConfirmedCases
0,Thailand,Thailand,15.0000,101.0000,01/22/2020,2
1,Japan,Japan,36.0000,138.0000,01/22/2020,2
2,Singapore,Singapore,1.2833,103.8333,01/22/2020,0
3,Nepal,Nepal,28.1667,84.2500,01/22/2020,0
4,Malaysia,Malaysia,2.5000,112.5000,01/22/2020,0


In [184]:
deaths.head()

,Province_State,Country_Region,Lat,Long,Date,Fatalities
0,Thailand,Thailand,15.0000,101.0000,01/22/2020,0
1,Japan,Japan,36.0000,138.0000,01/22/2020,0
2,Singapore,Singapore,1.2833,103.8333,01/22/2020,0
3,Nepal,Nepal,28.1667,84.2500,01/22/2020,0
4,Malaysia,Malaysia,2.5000,112.5000,01/22/2020,0


In [185]:
recovered.head()

,Province_State,Country_Region,Lat,Long,Date,RecoveredCases
0,Thailand,Thailand,15.0000,101.0000,01/22/2020,0
1,Japan,Japan,36.0000,138.0000,01/22/2020,0
2,Singapore,Singapore,1.2833,103.8333,01/22/2020,0
3,Nepal,Nepal,28.1667,84.2500,01/22/2020,0
4,Malaysia,Malaysia,2.5000,112.5000,01/22/2020,0


In [186]:
# merge confirmed, deaths and recovered
merged = pd.DataFrame()
for df, col in zip([confirmed, deaths, recovered], ['ConfirmedCases', 'Fatalities', 'RecoveredCases']):
    merged = df if len(merged)==0 else pd.concat([merged, df[col]], axis=1)

In [187]:
merged.head()

,Province_State,Country_Region,Lat,Long,Date,ConfirmedCases,Fatalities,RecoveredCases
0,Thailand,Thailand,15.0000,101.0000,01/22/2020,2,0,0
1,Japan,Japan,36.0000,138.0000,01/22/2020,2,0,0
2,Singapore,Singapore,1.2833,103.8333,01/22/2020,0,0,0
3,Nepal,Nepal,28.1667,84.2500,01/22/2020,0,0,0
4,Malaysia,Malaysia,2.5000,112.5000,01/22/2020,0,0,0


In [188]:
all_count = wide_to_long(merged, col='Count', pos=5, var_name='Type')

In [189]:
all_count.head()

,Province_State,Country_Region,Lat,Long,Date,Type,Count
0,Thailand,Thailand,15.0000,101.0000,01/22/2020,ConfirmedCases,2
1,Japan,Japan,36.0000,138.0000,01/22/2020,ConfirmedCases,2
2,Singapore,Singapore,1.2833,103.8333,01/22/2020,ConfirmedCases,0
3,Nepal,Nepal,28.1667,84.2500,01/22/2020,ConfirmedCases,0
4,Malaysia,Malaysia,2.5000,112.5000,01/22/2020,ConfirmedCases,0


In [190]:
def write_data(df, filename='covid19.csv', exclude_cols=['Lat', 'Long']):
    df.drop(exclude_cols, axis=1).to_csv(filename, index=False)

In [191]:
date=pd.to_datetime(confirmed['Date'].max()).strftime('%m.%d.%y')
write_data(confirmed, 'covid19-confirmed-{}.csv'.format(date))
write_data(deaths, 'covid19-deaths-{}.csv'.format(date))
write_data(recovered, 'covid19-recovered-{}.csv'.format(date))
write_data(merged, 'covid19-merged-{}.csv'.format(date))
write_data(all_count, 'covid19-{}.csv'.format(date))

In [192]:
def write_geo(data, filename='covid19.c6p', cols=['Province_State', 'Lat', 'Long']):
    data[cols].to_csv(filename, index=False)

In [193]:
write_geo(confirmed, 'covid19.c6p')